# Google cirqのチュートリアル


https://github.com/quantumlib/Cirq

## インストール

```
pip install --upgrade pip
pip install cirq
```

### とりあえず動かしてみる

In [238]:
import cirq

q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(0, 1)

circuit = cirq.Circuit()
circuit.append(cirq.H(q0))
circuit.append(cirq.CNOT(q0,q1))
circuit.append(cirq.RotZGate(half_turns = 0.25).on(q0))
circuit.append(cirq.H(q0))
circuit.append(cirq.measure(q0,q1, key='m'))

print("Circuit:")
print(circuit)

simulator = cirq.google.XmonSimulator()
result = simulator.run(circuit, repetitions=100)
print("Results:")
print(result.histogram(key='m'))


#cirq.Circuit.from_ops(
#    cirq.X(qubit)**0.5,  # Square root of NOT.
#    cirq.measure(qubit, key='m')  # Measurement.
#)



# Simulate the circuit several times.


Circuit:
(0, 0): ───H───@───T───H───M('m')───
               │           │
(0, 1): ───────X───────────M────────
Results:
Counter({0: 33, 3: 29, 1: 23, 2: 15})


### 量子ビット配列を定義する
cirqでは正方格子上の量子ビットを配置する `GridQubit` が定義されている。

In [239]:
#  縦と横の長さが length の量子ビット配列を定義する。
length = 3
# define qubits on the grid.
qubits = [cirq.GridQubit(i, j) for i in range(length) for j in range(length)]
print(qubits)
# prints 
# [GridQubit(0, 0), GridQubit(0, 1), GridQubit(0, 2), GridQubit(1, 0), GridQubit(1, 1), GridQubit(1, 2), GridQubit(2, 0), GridQubit(2, 1), GridQubit(2, 2)]

[GridQubit(0, 0), GridQubit(0, 1), GridQubit(0, 2), GridQubit(1, 0), GridQubit(1, 1), GridQubit(1, 2), GridQubit(2, 0), GridQubit(2, 1), GridQubit(2, 2)]


量子ビットの呼び方とグリッド上の位置

In [21]:
print(qubits[0])
print(qubits[1])
print(qubits[3])
print(qubits[8])

(0, 0)
(0, 1)
(1, 0)
(2, 2)


In [35]:
q0 = cirq.GridQubit(0,0)
print(q0)

(0, 0)


### 量子回路を構成する
空の量子回路を、 `Circuit`で用意する。量子回路に演算を追加する場合は、`append`を用いて量子演算を追加していく。量子回路を表示する場合は`print(circuit)`。量子演算は`Gate`で定義され、 `Gate.on`、もしくは、`Gate(作用する量子ビット)`で、どの量子ビットに作用するかを指定して、`GateOperation`に変換される。

In [36]:
# 空の量子回路を用意。
circuit = cirq.Circuit()
circuit.append(cirq.H.on(q) for q in qubits if (q.row + q.col) % 2 == 0)
circuit.append(cirq.X.on(q) for q in qubits if (q.row + q.col) % 2 == 1)
print(circuit)

(0, 0): ───H───────

(0, 1): ───────X───

(0, 2): ───H───────

(1, 0): ───────X───

(1, 1): ───H───────

(1, 2): ───────X───

(2, 0): ───H───────

(2, 1): ───────X───

(2, 2): ───H───────


In [37]:
# Gate
x_gate = cirq.X
print(X_gate)

# Gate を GateOperation に変換
x_ope = x_gate(qubits[0])
print(x_ope)

X
X((0, 0))


`circuit`は同時に実行可能な`GateOperation`の集合である`Moment`に分割されている。

In [38]:
for i, m in enumerate(circuit):
    print('Moment {}: {}'.format(i, m))
# prints 
# Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
# Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))

Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))


### 量子演算の種類
一般的な演算`CommonGates`とgoogleが開発するXmonに特化した演算、`XmonGates`がある。ここでは主に前者について説明する。

In [32]:
x_gate = cirq.X
y_gate = cirq.Y
z_gate = cirq.Z

h_gate = cirq.H
t_gate = cirq.T
s_gate = cirq.S

cz_gate = cirq.CZ
cnot_gate = cirq.CNOT
swap_gate = cirq.SWAP

circuit = cirq.Circuit()
circuit.append(x_gate(qubits[0]))
circuit.append(y_gate(qubits[0]))
circuit.append(z_gate(qubits[0]))
circuit.append(h_gate(qubits[0]))
circuit.append(t_gate(qubits[0]))
circuit.append(s_gate(qubits[0]))
circuit.append(cnot_gate(qubits[0],qubits[1]))
circuit.append(swap_gate(qubits[0],qubits[1]))
circuit.append(cz_gate(qubits[0],qubits[1]))

circuit.append(cirq.X.on(qubits[0]))
circuit.append(cirq.Y.on(qubits[0]))
circuit.append(cirq.Z.on(qubits[0]))
circuit.append(cirq.H.on(qubits[0]))
circuit.append(cirq.T.on(qubits[0]))
circuit.append(cirq.S.on(qubits[0]))
circuit.append(cirq.CNOT.on(qubits[0],qubits[1]))
circuit.append(cirq.SWAP.on(qubits[0],qubits[1]))
circuit.append(cirq.CZ.on(qubits[0],qubits[1]))
print(circuit)

(0, 0): ───X───Y───Z───H───T───S───@───×───@───X───Y───Z───H───T───S───@───×───@───
                                   │   │   │                           │   │   │
(0, 1): ───────────────────────────X───×───@───────────────────────────X───×───@───


連続的な演算。パラメータの入れ方は `half_turns`（X^a）のaを代入、`rads`ラジアン、`degs`度。

In [51]:
import math
# X
x_rot_gate = cirq.RotXGate(half_turns = 1)
print(x_rot_gate.matrix())

# square root of X
x_rot_gate = cirq.RotXGate(half_turns = 0.5)
print(x_rot_gate.matrix())

# Identity
x_rot_gate = cirq.RotXGate(half_turns = 0)
print(x_rot_gate.matrix())

# radian
x_rot_gate = cirq.RotXGate(rads = math.pi/2.0)
print(x_rot_gate.matrix())

# degree
# radian
x_rot_gate = cirq.RotXGate(degs = 90)
print(x_rot_gate.matrix())

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]


同様に `RotYGate`、`RotZGate`が利用可能。

また、|11>に位相を作用させるような `Rot11Gate`も利用可能。

In [55]:
rot11_gate = cirq.Rot11Gate(rads = math.pi/2.0)
print(rot11_gate.matrix())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j]]


その他に量子ビットAに対してのA^aのような連続演算を定義するものとして、`CNotGate`、`SwapGate`、`ISwapGate`が利用できる。

３量子ビットは、`CCZ`、`CCX`、`CSWAP`、`TOFFOLI`（CCXと同じ）、`FREDKIN`が利用可能。また、`XmonGate`として、

In [155]:
circuit = cirq.Circuit()

sqrtx_gate = cirq.RotXGate(half_turns = 0.5)
sqrty_gate = cirq.RotYGate(half_turns = 0.5)
sqrtz_gate = cirq.RotZGate(rads = math.pi/6.0)
rot11_gate = cirq.Rot11Gate(rads = math.pi/2.0)
circuit.append(h_gate(q) for q in  qubits)

circuit.append(x_gate(qubits[0]))
circuit.append(y_gate(qubits[0]))
circuit.append(z_gate(qubits[0]))
circuit.append(h_gate(qubits[0]))
circuit.append(t_gate(qubits[0]))
circuit.append(s_gate(qubits[0]))
circuit.append(cnot_gate(qubits[0],qubits[1]))
circuit.append(swap_gate(qubits[0],qubits[1]))
circuit.append(cz_gate(qubits[0],qubits[1]))
circuit.append(sqrtx_gate(qubits[2]))
circuit.append(sqrty_gate(qubits[2]))
circuit.append(sqrtz_gate(qubits[2]))
circuit.append(rot11_gate(qubits[2],qubits[3]))

print(circuit)


(0, 0): ───H───X───Y───Z───H───T───S───@───×───@─────────────────────────────────────
                                       │   │   │
(0, 1): ───H───────────────────────────X───×───@─────────────────────────────────────

(0, 2): ───H───────────────────────────────────────X^0.5───Y^0.5───Z^0.167───@───────
                                                                             │
(1, 0): ───H─────────────────────────────────────────────────────────────────@^0.5───

(1, 1): ───H─────────────────────────────────────────────────────────────────────────

(1, 2): ───H─────────────────────────────────────────────────────────────────────────

(2, 0): ───H─────────────────────────────────────────────────────────────────────────

(2, 1): ───H─────────────────────────────────────────────────────────────────────────

(2, 2): ───H─────────────────────────────────────────────────────────────────────────


### 測定を追加する

測定は `Circuit` に `MesurementGate`を追加する。

In [156]:
meas = cirq.MeasurementGate(key = 'x')
meas_ope = meas.on(qubits[0])

# `Circuit` のdeep copy 
circuit_with_meas = cirq.Circuit.copy(circuit)
circuit_with_meas.append(meas_ope)
print(circuit_with_meas)

(0, 0): ───H───X───Y───Z───H───T───S───@───×───@─────────────────────────────────────M('x')───
                                       │   │   │
(0, 1): ───H───────────────────────────X───×───@──────────────────────────────────────────────

(0, 2): ───H───────────────────────────────────────X^0.5───Y^0.5───Z^0.167───@────────────────
                                                                             │
(1, 0): ───H─────────────────────────────────────────────────────────────────@^0.5────────────

(1, 1): ───H──────────────────────────────────────────────────────────────────────────────────

(1, 2): ───H──────────────────────────────────────────────────────────────────────────────────

(2, 0): ───H──────────────────────────────────────────────────────────────────────────────────

(2, 1): ───H──────────────────────────────────────────────────────────────────────────────────

(2, 2): ───H──────────────────────────────────────────────────────────────────────────────────


もしくは、

In [157]:
circuit_with_meas = cirq.Circuit.copy(circuit)
circuit_with_meas.append(cirq.measure(qubits[0],key='x'))
print(circuit_with_meas)

(0, 0): ───H───X───Y───Z───H───T───S───@───×───@─────────────────────────────────────M('x')───
                                       │   │   │
(0, 1): ───H───────────────────────────X───×───@──────────────────────────────────────────────

(0, 2): ───H───────────────────────────────────────X^0.5───Y^0.5───Z^0.167───@────────────────
                                                                             │
(1, 0): ───H─────────────────────────────────────────────────────────────────@^0.5────────────

(1, 1): ───H──────────────────────────────────────────────────────────────────────────────────

(1, 2): ───H──────────────────────────────────────────────────────────────────────────────────

(2, 0): ───H──────────────────────────────────────────────────────────────────────────────────

(2, 1): ───H──────────────────────────────────────────────────────────────────────────────────

(2, 2): ───H──────────────────────────────────────────────────────────────────────────────────


まとめて追加するなら、

In [158]:
circuit_with_meas = cirq.Circuit.copy(circuit)
circuit_with_meas.append(cirq.measure(*qubits,key='x'))
print(circuit_with_meas)

(0, 0): ───H───X───Y───Z───H───T───S───@───×───@─────────────────────────────────────M('x')───
                                       │   │   │                                     │
(0, 1): ───H───────────────────────────X───×───@─────────────────────────────────────M────────
                                                                                     │
(0, 2): ───H───────────────────────────────────────X^0.5───Y^0.5───Z^0.167───@───────M────────
                                                                             │       │
(1, 0): ───H─────────────────────────────────────────────────────────────────@^0.5───M────────
                                                                                     │
(1, 1): ───H─────────────────────────────────────────────────────────────────────────M────────
                                                                                     │
(1, 2): ───H─────────────────────────────────────────────────────────────────────────M────

### 構成した回路を実行する（シミュレータ）

In [169]:
simulator = cirq.google.XmonSimulator()
#もしくは from cirq.google import XmonSimulator

result = simulator.run(circuit_with_meas,repetitions=1, qubit_order=qubits)
print(result)

result = simulator.simulate(circuit_with_meas, qubit_order=qubits)
print(result)

x=1, 0, 1, 1, 0, 0, 0, 0, 0
x=101010101


何回も繰り返したヒストグラムは、


In [161]:
result = simulator.run(circuit_with_meas,repetitions=1000, qubit_order=qubits)
print(result.histogram(key='x'))

Counter({380: 16, 102: 14, 112: 13, 101: 13, 109: 12, 111: 12, 358: 12, 124: 12, 351: 12, 353: 12, 105: 12, 114: 12, 345: 12, 327: 11, 328: 11, 87: 11, 121: 11, 338: 11, 72: 11, 70: 11, 369: 11, 354: 11, 76: 11, 93: 10, 85: 10, 119: 10, 324: 10, 373: 10, 98: 10, 375: 10, 103: 10, 67: 10, 113: 10, 341: 10, 64: 10, 95: 9, 371: 9, 360: 9, 339: 9, 366: 9, 382: 9, 84: 9, 363: 9, 326: 9, 372: 9, 122: 9, 107: 9, 364: 9, 82: 9, 381: 9, 108: 8, 86: 8, 322: 8, 71: 8, 66: 8, 330: 8, 368: 8, 355: 8, 359: 8, 79: 8, 110: 8, 74: 8, 332: 8, 325: 8, 337: 8, 126: 8, 333: 8, 356: 8, 127: 8, 323: 8, 347: 7, 352: 7, 346: 7, 97: 7, 89: 7, 118: 7, 370: 7, 344: 7, 342: 7, 378: 7, 117: 7, 336: 7, 320: 7, 77: 7, 104: 7, 81: 7, 383: 7, 120: 7, 329: 6, 376: 6, 350: 6, 106: 6, 78: 6, 115: 6, 334: 6, 362: 6, 99: 6, 377: 6, 96: 6, 83: 6, 125: 6, 357: 5, 69: 5, 94: 5, 91: 5, 73: 5, 374: 5, 68: 5, 123: 5, 116: 5, 75: 5, 361: 5, 379: 4, 365: 4, 349: 4, 80: 4, 88: 4, 90: 4, 335: 4, 92: 3, 340: 3, 331: 3, 321: 3, 343: 3,

例えば、380:16は、測定結果のビット列を整数で表示したとき380に対応するものが16回カウントされた。今は9量子ビットの測定結果９ビットで表現される0~511までの整数のカウント数が表示されている。

In [170]:
result = simulator.simulate(circuit_with_meas, qubit_order=qubits)
print(result)

x=101001000


### 1量子ビット演算の速度計測

In [208]:
import time

for k in range(16):
    length = 5+k
    qubits = [cirq.GridQubit(i, 0) for i in range(length)]
    circuit = cirq.Circuit()
    depth = 2
    for i in range(depth):
        circuit.append(sqrtx_gate(q) for q in qubits)
    #print(circuit)

    start = time.time()
    result = simulator.simulate(circuit)
    elapsed_time = time.time() - start
    print("{0} ".format(length) + "{0}".format(elapsed_time/depth/length))


5 0.0005863189697265625
6 0.0004938443501790365
7 0.00038863931383405415
8 0.00035788118839263916
9 0.0003858274883694119
10 0.0003578424453735352
11 0.00038905577226118606
12 0.0004514952500661214
13 0.0006179259373591497
14 0.0009067228862217494
15 0.0014355977376302083
16 0.002659842371940613
17 0.006418326321770163
18 0.009363724125756158
19 0.012215106110823782
20 0.02017190456390381


### ２量子ビットの速度計測

In [218]:
for k in range(15):
    length = 5+k
    qubits = [cirq.GridQubit(i, 0) for i in range(length)]
    circuit = cirq.Circuit()

    num_gates =0;
    for i in range(depth):
        for q in range(length-1):
            for p in range(q):
                circuit.append(cirq.CNOT(qubits[q],qubits[p]))
                num_gates = num_gates +1
    #print(circuit)
    #print(num_gates)
    start = time.time()
    result = simulator.simulate(circuit)
    elapsed_time = time.time() - start
    print("{0} ".format(length) + "{0}".format(elapsed_time/num_gates))

5 0.005272487799326579
6 0.005054450035095215
7 0.006153702735900879
8 0.005717947369530087
9 0.005064236266272408
10 0.005918873680962456
11 0.005739357736375596
12 0.0058295986869118434
13 0.0063888340285330105
14 0.007797409326602251
15 0.011585434714516441
16 0.017545210747491744
17 0.032306812206904095
18 0.06696187485666837
19 0.08677310803357292
